#### Beyond text classification: Sequence-to-sequence learning
You now possess all of the tools you will need to tackle most natural language processing tasks. However, you’ve only seen these tools in action on a single problem: text classification. This is an extremely popular use case, but there’s a lot more to NLP than classification. In this section, you’ll deepen your expertise by learning about **sequence-to-sequence models**. <br>
A sequence-to-sequence model takes a sequence as input (often a sentence or paragraph) and translates it into a different sequence. This is the task at the heart of many of the most successful applications of NLP:
- **Machine translation**—Convert a paragraph in a source language to its equivalent in a target language.
- **Text summarization**—Convert a long document to a shorter version that retains the most important information.
- **Question answering**—Convert an input question into its answer.
- **Chatbots**—Convert a dialogue prompt into a reply to this prompt, or convert the history of a conversation into the next reply in the conversation.
- **Text generation**—Convert a text prompt into a paragraph that completes the prompt.
- Etc.

The general template behind sequence-to-sequence models is described in figure 11.12. <br>
**During training**,
- An **encoder** model turns the source sequence into an intermediate representation.
- A **decoder** is trained to predict the next token i in the target sequence by looking at both previous tokens (0 to i - 1) and the encoded source sequence.

![](./images/11.12.png)

**During inference**, we don’t have access to the target sequence—we’re trying to predict it from scratch. We’ll have to generate it one token at a time:
1. We obtain the encoded source sequence from the encoder.
2. The decoder starts by looking at the encoded source sequence as well as an initial “seed” token (such as the string "[start]"), and uses them to predict the first real token in the sequence.
3. The predicted sequence so far is fed back into the decoder, which generates the next token, and so on, until it generates a stop token (such as the string "[end]").

Everything you’ve learned so far can be repurposed to build this new kind of model. <br>
Let’s dive in.

##### A machine translation example
We’ll demonstrate **sequence-to-sequence** modeling on a **machine translation** task. Machine translation is precisely what Transformer was developed for! We’ll start with a recurrent sequence model, and we’ll follow up with the full **Transformer** architecture. <br>
We’ll be working with an English-to-Spanish translation dataset available at http://www.manythings.org/anki/ . Let’s download it:

In [1]:
!wget http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip

In [2]:
!unzip -q spa-eng.zip

The text file contains one example per line: an English sentence, followed by a tab character, followed by the corresponding Spanish sentence. Let’s parse this file.

In [3]:
text_file = "spa-eng/spa.txt"
with open(text_file, encoding="utf8") as f:
    lines = f.read().split("\n")[:-1]
text_pairs = []
for line in lines: # Iterate over the lines in the file.
    english, spanish = line.split("\t") # Each line contains an English phrase and its Spanish translation, tab-separated.
    spanish = "[start]" + spanish + "[end]" # We prepend "[start]" and append "[end]" to the Spanish sentence, to match the template from figure 11.12.
    text_pairs.append((english, spanish))

Our text_pairs look like this:

In [4]:
import random

print(random.choice(text_pairs))

Let’s shuffle them and split them into the usual training, validation, and test sets:

In [5]:
random.shuffle(text_pairs)
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples:num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples:]

Next, let’s prepare two separate **TextVectorization** layers: one for English and one for Spanish. We’re going to need to customize the way strings are preprocessed:
- We need to preserve the "[start]" and "[end]" tokens that we’ve inserted. By default, the characters [ and ] would be stripped, but we want to keep them around so we can tell apart the word “start” and the start token "[start]".
- Punctuation is different from language to language! In the Spanish **TextVectorization** layer, if we’re going to strip punctuation characters, we need to also strip the character ¿.

Note that for a non-toy translation model, we would treat punctuation characters as separate tokens rather than stripping them, since we would want to be able to generate correctly punctuated sentences. In our case, for simplicity, we’ll get rid of all punctuation.

##### Vectorizing the English and Spanish text pairs

In [6]:
import tensorflow as tf
from tensorflow.keras import layers
import string
import re

# Prepare a custom string standardization function for the Spanish TextVectorization layer: 
# it preserves [ and ] but strips ¿ (as well as all other characters from strings.punctuation).
strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")

def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(lowercase, f"[{re.escape(strip_chars)}]", "")

# To keep things simple, we’ll only look at the top 15,000 words in each language, and we’ll restrict sentences to 20 words.
vocab_size = 15000
sequence_length = 20

# The English layer
source_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length,
)

# The Spanish layer
target_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length + 1, # Generate Spanish sentences that have one extra token, since we’ll need to offset the sentence by one step during training.
    standardize=custom_standardization,
)

train_english_texts = [pair[0] for pair in train_pairs]
train_spanish_texts = [pair[1] for pair in train_pairs]

# Learn the vocabulary of each language.
source_vectorization.adapt(train_english_texts)
target_vectorization.adapt(train_spanish_texts)

Finally, we can turn our data into a **tf.data** pipeline. We want it to return a tuple (inputs, target) where **inputs** is a dict with two keys, “encoder_inputs” (the English sentence) and “decoder_inputs” (the Spanish sentence), and **target** is the Spanish sentence offset by one step ahead.
##### Preparing datasets for the translation task

In [7]:
batch_size = 64

def format_dataset(eng, spa):
    eng = source_vectorization(eng)
    spa = target_vectorization(spa)
    return ({
        "english": eng,
        "spanish": spa[:, :-1], # The input Spanish sentence doesn’t include the last token to keep inputs and targets at the same length.
    }, spa[:, 1:]) # The target Spanish sentence is one step ahead. Both are still the same length (20 words).

def make_dataset(pairs):
    eng_texts, spa_texts = zip(*pairs)
    eng_texts = list(eng_texts)
    spa_texts = list(spa_texts)
    dataset = tf.data.Dataset.from_tensor_slices((eng_texts, spa_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset, num_parallel_calls=4)
    return dataset.shuffle(2048).prefetch(16).cache() # Use in-memory caching to speed up preprocessing.

train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

Here’s what our dataset outputs look like:

In [8]:
for inputs, targets in train_ds.take(1):
    print(f"inputs['english'].shape: {inputs['english'].shape}")
    print(f"inputs['spanish'].shape: {inputs['spanish'].shape}")
    print(f"targets.shape: {targets.shape}")

The data is now ready—time to build some models. We’ll start with a recurrent sequence-to-sequence model before moving on to a Transformer.

##### Sequence-to-sequence learning with RNNs

**Recurrent neural networks** dominated sequence-to-sequence learning from 2015–2017 before being overtaken by **Transformer**. They were the basis for many real world machine-translation systems—as mentioned in chapter 10, Google Translate circa 2017 was powered by a stack of seven large LSTM layers. It’s still worth learning about this approach today, as it provides an easy entry point to understanding sequence-to-sequence models. <br>
The simplest, naive way to use RNNs to turn a sequence into another sequence is to keep the output of the RNN at each time step. In Keras, it would look like this:

```python
inputs = keras.Input(shape=(sequence_length,), dtype="int64")
x = layers.Embedding(input_dim=vocab_size, output_dim=128)(inputs)
x = layers.LSTM(32, return_sequences=True)(x)
outputs = layers.Dense(vocab_size, activation="softmax")(x)
model = keras.Model(inputs, outputs)
```

However, there are two major issues with this approach:
- The target sequence must always be the same length as the source sequence. In practice, this is rarely the case. Technically, this isn’t critical, as you could always pad either the source sequence or the target sequence to make their lengths match.
- Due to the step-by-step nature of RNNs, the model will only be looking at tokens 0…N in the source sequence in order to predict token N in the target sequence. This constraint makes this setup unsuitable for most tasks, and particularly translation. Consider translating “The weather is nice today” to French—that would be “Il fait beau aujourd’hui.” You’d need to be able to predict “Il” from just “The,” “Il fait” from just “The weather,” etc., which is simply impossible.

If you’re a human translator, you’d start by reading the entire source sentence before starting to translate it. This is especially important if you’re dealing with languages that have wildly different word ordering, like English and Japanese. And that’s exactly what standard sequence-to-sequence models do. <br>
In a proper sequence-to-sequence setup (see figure 11.13), you would first use an RNN (**the encoder**) to turn the entire source sequence into a single vector (or set of vectors). This could be the last output of the RNN, or alternatively, its final internal state vectors. Then you would use this vector (or vectors) as the initial state of another RNN (**the decoder**), which would look at elements 0…N in the target sequence, and try to predict step N+1 in the target sequence.

![](./images/11.13.png)

Let’s implement this in Keras with **GRU-based encoders and decoders**. The choice of GRU rather than LSTM makes things a bit simpler, since **GRU** only has a single state vector, whereas **LSTM** has multiple. <br>
Let’s start with the encoder.

##### GRU-based encoder

In [9]:
from tensorflow import keras
from tensorflow.keras import layers

embed_dim = 256
latent_dim = 1024

# The English source sentence goes here.
# Specifying the name of the input enables us to fit() the model with a dict of inputs.
source = keras.Input(shape=(None,), dtype="int64", name="english")
# Don’t forget masking: it’s critical in this setup.
x = layers.Embedding(vocab_size, embed_dim, mask_zero=True)(source)
# Our encoded source sentence is the last output of a bidirectional GRU.
encoded_source = layers.Bidirectional(layers.GRU(latent_dim), merge_mode="sum")(x)

Next, let’s add the decoder—a simple GRU layer that takes as its initial state the encoded source sentence. On top of it, we add a Dense layer that produces for each output step a probability distribution over the Spanish vocabulary.

##### GRU-based decoder and the end-to-end model

In [10]:
past_target = keras.Input(shape=(None,), dtype="int64", name="spanish") # The Spanish target sentence goes here.
x = layers.Embedding(vocab_size, embed_dim, mask_zero=True)(past_target) # Don’t forget masking.
decoder_gru = layers.GRU(latent_dim, return_sequences=True)
# The encoded source sentence serves as the initial state of the decoder GRU.
x = decoder_gru(x, initial_state=encoded_source)
x = layers.Dropout(0.5)(x)
target_next_step = layers.Dense(vocab_size, activation="softmax")(x) # Predicts the next token.
# End-to-end model: maps the source sentence and the target sentence to the target sentence one step in the future
seq2seq_rnn = keras.Model(inputs=[source, past_target], outputs=target_next_step)

During training, the decoder takes as input the entire target sequence, but thanks to the step-by-step nature of RNNs, it only looks at tokens 0…N in the input to predict token N in the output (which corresponds to the next token in the sequence, since the output is intended to be offset by one step). This means we only use information from the past to predict the future, as we should; otherwise we’d be cheating, and our model would not work at inference time. <br>
Let’s start training.

##### Training our recurrent sequence-to-sequence model

In [11]:
seq2seq_rnn.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"])

seq2seq_rnn.fit(train_ds, epochs=15, validation_data=val_ds)

We picked accuracy as a crude way to monitor validation-set performance during training. We get to 64% accuracy: on average, the model predicts the next word in the Spanish sentence correctly 64% of the time. However, in practice, next-token accuracy isn’t a great metric for machine translation models, in particular because it makes the assumption that the correct target tokens from 0 to N are already known when predicting token N+1. In reality, during inference, you’re generating the target sentence from scratch, and you can’t rely on previously generated tokens being 100% correct. If you work on a real-world machine translation system, you will likely use **“BLEU scores”** to evaluate your models—a metric that looks at entire generated sequences and that seems to correlate well with human perception of translation quality. <br>
At last, let’s use our model for inference. We’ll pick a few sentences in the test set and check how our model translates them. We’ll start from the seed token, "[start]", and feed it into the decoder model, together with the encoded English source sentence. We’ll retrieve a next-token prediction, and we’ll re-inject it into the decoder repeatedly, sampling one new target token at each iteration, until we get to "[end]" or reach the maximum sentence length.

##### Translating new sentences with our RNN encoder and decoder

Note that this inference setup, while very simple, is rather inefficient, since we reprocess the entire source sentence and the entire generated target sentence every time we sample a new word. In a practical application, you’d factor the encoder and the decoder as two separate models, and your decoder would only run a single step at
each token-sampling iteration, reusing its previous internal state. <br>
Here are our translation results. Our model works decently well for a toy model, though it still makes many basic mistakes.

##### Some sample results from the recurrent translation model

In [12]:
import numpy as np

spa_vocab = target_vectorization.get_vocabulary()
# Prepare a dict to convert token index predictions to string tokens.
spa_index_lookup = dict(zip(range(len(spa_vocab)), spa_vocab))
max_decoded_sentence_length = 20

def decode_sequence(input_sentence):
    tokenized_input_sentence = source_vectorization([input_sentence])
    decoded_sentence = "[start]" # Seed token
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = target_vectorization([decoded_sentence])
        # Sample the next token.
        next_token_predictions = seq2seq_rnn.predict(
            [tokenized_input_sentence, tokenized_target_sentence])
        sampled_token_index = np.argmax(next_token_predictions[0, i, :])
        # Convert the next token prediction to a string and append it to the generated sentence.
        sampled_token = spa_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token
        # Exit condition: either hit max length or sample a stop character
        if sampled_token == "[end]":
            break
    return decoded_sentence

test_eng_texts = [pair[0] for pair in test_pairs]
for _ in range(20):
    input_sentence = random.choice(test_eng_texts)
    print("-")
    print(input_sentence)
    print(decode_sequence(input_sentence))

There are many ways this toy model could be improved: We could use a deep stack of recurrent layers for both the encoder and the decoder (note that for the decoder, this makes state management a bit more involved). We could use an LSTM instead of a GRU. And so on. Beyond such tweaks, however, the RNN approach to sequence-to-sequence learning has a few fundamental limitations:
- The source sequence representation has to be held entirely in the encoder state vector(s), which puts significant limitations on the size and complexity of the sentences you can translate. It’s a bit as if a human were translating a sentence entirely from memory, without looking twice at the source sentence while producing the translation.
- RNNs have trouble dealing with very long sequences, since they tend to progressively forget about the past—by the time you’ve reached the 100th token in either sequence, little information remains about the start of the sequence. That means RNN-based models can’t hold onto long-term context, which can be essential for translating long documents.

These limitations are what has led the machine learning community to embrace the Transformer architecture for sequence-to-sequence problems. Let’s take a look.

#### Sequence-to-sequence learning with Transformer
Sequence-to-sequence learning is the task where Transformer really shines. Neural attention enables Transformer models to successfully process sequences that are considerably longer and more complex than those RNNs can handle. <br>
As a human translating English to Spanish, you’re not going to read the English sentence one word at a time, keep its meaning in memory, and then generate the Spanish sentence one word at a time. That may work for a five-word sentence, but it’s unlikely to work for an entire paragraph. Instead, you’ll probably want to go back and forth between the source sentence and your translation in progress, and pay attention to different words in the source as you’re writing down different parts of your translation. <br>
That’s exactly what you can achieve with neural attention and Transformers. You’re already familiar with the **Transformer encoder**, which uses self-attention to produce context-aware representations of each token in an input sequence. In a sequence-to-sequence Transformer, the Transformer encoder would naturally play the role of the encoder, which reads the source sequence and produces an encoded representation of it. Unlike our previous RNN encoder, though, the Transformer encoder keeps the encoded representation in a sequence format: it’s a sequence of context-aware embedding vectors.

The second half of the model is the **Transformer decoder**. Just like the RNN decoder, it reads tokens 0…N in the target sequence and tries to predict token N+1. Crucially, while doing this, it uses neural attention to identify which tokens in the encoded source sentence are most closely related to the target token it’s currently trying to predict—perhaps not unlike what a human translator would do. Recall the query-key-value model: in a Transformer decoder, the target sequence serves as an attention “query” that is used to to pay closer attention to different parts of the source sequence (the source sequence plays the roles of both keys and values).

##### THE TRANSFORMER DECODER
Figure 11.14 shows the full sequence-to-sequence Transformer. Look at the **decoder** internals: you’ll recognize that it looks very similar to the **Transformer encoder**, except that an extra attention block is inserted between the **self-attention block** applied to the target sequence and the dense layers of the exit block. <br>
Let’s implement it. Like for the **TransformerEncoder**, we’ll use a **Layer** subclass. Before we focus on the **call()**, method, where the action happens, let’s start by defining the class constructor, containing the layers we’re going to need.

![](./images/11.14.png)

The **call()** method is almost a straightforward rendering of the connectivity diagram from figure 11.14. But there’s an additional detail we need to take into account: **causal padding**. Causal padding is absolutely critical to successfully training a sequence-to-sequence Transformer. Unlike an RNN, which looks at its input one step at a time, and thus will only have access to steps 0...N to generate output step N (which is token N+1 in the target sequence), the **TransformerDecoder** is order-agnostic: it looks at the entire target sequence at once. If it were allowed to use its entire input, it would simply learn to copy input step N+1 to location N in the output. The model would thus achieve perfect training accuracy, but of course, when running inference, it would be completely useless, since input steps beyond N aren’t available. <br>
The fix is simple: we’ll mask the upper half of the pairwise attention matrix to prevent the model from paying any attention to information from the future—only information from tokens 0...N in the target sequence should be used when generating target token N+1. To do this, we’ll add a **get_causal_attention_mask(self, inputs)** method to our **TransformerDecoder** to retrieve an attention mask that we can pass to our **MultiHeadAttention** layers.

In [15]:
class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        # This attribute ensures that the layer will propagate its input mask to its outputs; masking in Keras is explicitly opt-in. 
        # If you pass a mask to a layer that doesn’t implement compute_mask() and that doesn’t expose this supports_masking attribute, that’s an error.
        self.supports_masking = True

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        # Generate matrix of shape (sequence_length, sequence_length) with 1s in one half and 0s in the other.
        mask = tf.cast(i >= j, dtype="int32")
        # Replicate it along the batch axis to get a matrix of shape (batch_size, sequence_length, sequence_length).
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1),
             tf.constant([1, 1], dtype=tf.int32)], axis=0)
        return tf.tile(mask, mult)

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs) # Retrieve the causal mask.
        # Prepare the input mask (that describes padding locations in the target sequence).
        if mask is not None:
            padding_mask = tf.cast(
                mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask) # Merge the two masks together.
        attention_output_1 = self.attention_1(
            query=inputs,
            value=inputs,
            key=inputs,
            attention_mask=causal_mask) # Pass the causal mask to the first attention layer, which performs self-attention over the target sequence.
        attention_output_1 = self.layernorm_1(inputs + attention_output_1)
        attention_output_2 = self.attention_2(
            query=attention_output_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask, # Pass the combined mask to the second attention layer, which relates the source sequence to the target sequence.
        )
        attention_output_2 = self.layernorm_2(
            attention_output_1 + attention_output_2)
        proj_output = self.dense_proj(attention_output_2)
        return self.layernorm_3(attention_output_2 + proj_output)

##### PUTTING IT ALL TOGETHER: A TRANSFORMER FOR MACHINE TRANSLATION
The **end-to-end Transformer** is the model we’ll be training. It maps the source sequence and the target sequence to the target sequence one step in the future. It straightforwardly combines the pieces we’ve built so far: **PositionalEmbedding** layers, the **TransformerEncoder**, and the **TransformerDecoder**. Note that both the TransformerEncoder and the TransformerDecoder are shape-invariant, so you could be stacking many of them to create a more powerful encoder or decoder. In our example, we’ll stick to a single instance of each.

##### PositionalEmbedding layer

In [16]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=input_dim, output_dim=output_dim)
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim)
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)

    def get_config(self):
        config = super(PositionalEmbedding, self).get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config

##### End-to-end Transformer

In [ ]:
embed_dim = 256
dense_dim = 2048
num_heads = 8

encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="english")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, dense_dim, num_heads)(x) # Encode the source sentence.

decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="spanish")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)
x = TransformerDecoder(embed_dim, dense_dim, num_heads)(x, encoder_outputs) # Encode the target sentence and combine it with the encoded source sentence.
x = layers.Dropout(0.5)(x)
decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x) # Predict a word for each output position.
transformer = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

##### Training the sequence-to-sequence Transformer

In [ ]:
transformer.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"])
transformer.fit(train_ds, epochs=30, validation_data=val_ds)

Finally, let’s try using our model to translate never-seen-before English sentences from the test set. The setup is identical to what we used for the sequence-to-sequence RNN model.

##### Translating new sentences with our Transformer model

In [ ]:
import numpy as np
spa_vocab = target_vectorization.get_vocabulary()
spa_index_lookup = dict(zip(range(len(spa_vocab)), spa_vocab))
max_decoded_sentence_length = 20

def decode_sequence(input_sentence):
    tokenized_input_sentence = source_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = target_vectorization(
            [decoded_sentence])[:, :-1]
        # Sample the next token.
        predictions = transformer(
            [tokenized_input_sentence, tokenized_target_sentence])
        sampled_token_index = np.argmax(predictions[0, i, :])
        # Convert the next token prediction to a string, and append it to the generated sentence.
        sampled_token = spa_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token
        if sampled_token == "[end]":
            break
    return decoded_sentence

test_eng_texts = [pair[0] for pair in test_pairs]
for _ in range(20):
    input_sentence = random.choice(test_eng_texts)
    print("-")
    print(input_sentence)
    print(decode_sequence(input_sentence))

Subjectively, the Transformer seems to perform significantly better than the GRU based translation model. It’s still a toy model, but it’s a better toy model.

That concludes this chapter on natural language processing—you just went from the very basics to a fully fledged Transformer that can translate from English to Spanish. Teaching machines to make sense of language is the latest superpower you can add to your collection.

#### Summary
- There are two kinds of NLP models: **bag-of-words models** that process sets of words or **N-grams** without taking into account their order, and **sequence models** that process word order. A bag-of-words model is made of Dense layers, while a sequence model could be an RNN, a 1D convnet, or a Transformer.
- When it comes to text classification, the ratio between the number of samples in your training data and the mean number of words per sample can help you determine whether you should use a bag-of-words model or a sequence model.
- **Word embeddings** are vector spaces where semantic relationships between words are modeled as distance relationships between vectors that represent those words.
- **Sequence-to-sequence learning** is a generic, powerful learning framework that can be applied to solve many NLP problems, including machine translation. A sequence-to-sequence model is made of an **encoder**, which processes a source sequence, and a **decoder**, which tries to predict future tokens in target sequence by looking at past tokens, with the help of the encoder-processed source sequence.
- **Neural attention** is a way to create context-aware word representations. It’s the basis for the **Transformer** architecture.
- The Transformer architecture, which consists of a TransformerEncoder and a TransformerDecoder, yields excellent results on sequence-to-sequence tasks. The first half, the TransformerEncoder, can also be used for text classification or any sort of single-input NLP task.